In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.dataset import random_split
from torchvision import datasets, transforms

# Training settings
b_size = 512
n_epochs = 500
lr = 0.01
valid_step = 100
patience = max_patience = 10
best_loss = 100000
model_file = './models/model2.md'
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)    

def test_validate(model, device, test_loader, test_valid='Test'):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\n{} set: Average loss: {:.6f}, Accuracy: {}/{} ({:.6f}%)\n'.format(
        test_valid, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss
    
def train(model, device, train_loader, valid_loader, optimizer, epoch):
    global best_loss, patience
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        if batch_idx % valid_step == 0:
            print('Start Validating...')
            valid_loss = test_validate(model, device, valid_loader, 'Valid')
            if valid_loss < best_loss:
                best_loss = valid_loss
                patience = max_patience
                print('Bese valid loss: {}'.format(best_loss))
                print('Improved! Reset patience.')
                print('Saving model...')
                torch.save(model, model_file)
            else:
                patience -= 1
                print('Not improved... Patience: {}'.format(patience))
                if patience == 0:
                    print('Out of patience. Stop training.')
                    return

def main():        
    dataroot = 'D:\Lab\dataset'

    device = torch.device("cuda")

    total_train_dataset = datasets.MNIST(dataroot, train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))
    total_train_size = len(total_train_dataset)
    split_ratio = 0.8
    train_size = int(split_ratio * total_train_size)
    valid_size = total_train_size - train_size
    
    train_dataset, valid_dataset = random_split(total_train_dataset,[train_size, valid_size] )
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=b_size, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=b_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(dataroot, train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=b_size, shuffle=True)


    model = Net().to(device)
    print(model)
    optimizer = optim.SGD(model.parameters(), lr=lr)

    for epoch in range(1, n_epochs + 1):
        train(model, device, train_loader, valid_loader, optimizer, epoch)
        if patience == 0:
            break
                
    print('Start testing...')
    model = torch.load(model_file)
    test_validate(model, device, test_loader)
        
main()

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)
Train Epoch: 1 [0/48000 (0%)]	Loss: 2.339640
Start Validating...

Valid set: Average loss: 2.305821, Accuracy: 1659/12000 (13.825000%)

Bese valid loss: 2.305820566813151
Improved! Reset patience.
Saving model...


C:\Users\Likit\Anaconda3\envs\tensorflow\lib\site-packages\torch\serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Train Epoch: 1 [25600/48000 (53%)]	Loss: 2.200782
Train Epoch: 2 [0/48000 (0%)]	Loss: 2.159272
Start Validating...

Valid set: Average loss: 2.003270, Accuracy: 4978/12000 (41.483333%)

Bese valid loss: 2.003269989013672
Improved! Reset patience.
Saving model...
Train Epoch: 2 [25600/48000 (53%)]	Loss: 1.426624
Train Epoch: 3 [0/48000 (0%)]	Loss: 2.671303
Start Validating...

Valid set: Average loss: 0.929586, Accuracy: 9233/12000 (76.941667%)

Bese valid loss: 0.9295863355000814
Improved! Reset patience.
Saving model...
Train Epoch: 3 [25600/48000 (53%)]	Loss: 0.610859
Train Epoch: 4 [0/48000 (0%)]	Loss: 3.822577
Start Validating...

Valid set: Average loss: 0.590454, Accuracy: 9842/12000 (82.016667%)

Bese valid loss: 0.5904537862141928
Improved! Reset patience.
Saving model...
Train Epoch: 4 [25600/48000 (53%)]	Loss: 0.493160
Train Epoch: 5 [0/48000 (0%)]	Loss: 3.125381
Start Validating...

Valid set: Average loss: 0.461593, Accuracy: 10306/12000 (85.883333%)

Bese valid loss: 0.461

Train Epoch: 35 [0/48000 (0%)]	Loss: 1.662156
Start Validating...

Valid set: Average loss: 0.110102, Accuracy: 11602/12000 (96.683333%)

Not improved... Patience: 9
Train Epoch: 35 [25600/48000 (53%)]	Loss: 0.124476
Train Epoch: 36 [0/48000 (0%)]	Loss: 1.747138
Start Validating...

Valid set: Average loss: 0.106961, Accuracy: 11602/12000 (96.683333%)

Not improved... Patience: 8
Train Epoch: 36 [25600/48000 (53%)]	Loss: 0.084682
Train Epoch: 37 [0/48000 (0%)]	Loss: 1.643848
Start Validating...

Valid set: Average loss: 0.108756, Accuracy: 11600/12000 (96.666667%)

Not improved... Patience: 7
Train Epoch: 37 [25600/48000 (53%)]	Loss: 0.057576
Train Epoch: 38 [0/48000 (0%)]	Loss: 1.683550
Start Validating...

Valid set: Average loss: 0.104911, Accuracy: 11616/12000 (96.800000%)

Bese valid loss: 0.10491099786758423
Improved! Reset patience.
Saving model...
Train Epoch: 38 [25600/48000 (53%)]	Loss: 0.060672
Train Epoch: 39 [0/48000 (0%)]	Loss: 1.749224
Start Validating...

Valid set: Ave


Valid set: Average loss: 0.069123, Accuracy: 11742/12000 (97.850000%)

Bese valid loss: 0.06912347475687662
Improved! Reset patience.
Saving model...
Train Epoch: 69 [25600/48000 (53%)]	Loss: 0.060972
Train Epoch: 70 [0/48000 (0%)]	Loss: 1.424445
Start Validating...

Valid set: Average loss: 0.071752, Accuracy: 11728/12000 (97.733333%)

Not improved... Patience: 9
Train Epoch: 70 [25600/48000 (53%)]	Loss: 0.043761
Train Epoch: 71 [0/48000 (0%)]	Loss: 1.319427
Start Validating...

Valid set: Average loss: 0.068739, Accuracy: 11734/12000 (97.783333%)

Bese valid loss: 0.06873859723409016
Improved! Reset patience.
Saving model...
Train Epoch: 71 [25600/48000 (53%)]	Loss: 0.031435
Train Epoch: 72 [0/48000 (0%)]	Loss: 1.509486
Start Validating...

Valid set: Average loss: 0.077664, Accuracy: 11708/12000 (97.566667%)

Not improved... Patience: 9
Train Epoch: 72 [25600/48000 (53%)]	Loss: 0.019418
Train Epoch: 73 [0/48000 (0%)]	Loss: 1.656738
Start Validating...

Valid set: Average loss: 0.07


Test set: Average loss: 0.047285, Accuracy: 9845/10000 (98.450000%)

